In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
train_data=pd.read_csv('/kaggle/input/deepweedsx/train_set_labels.csv')
test_data= pd.read_csv('/kaggle/input/deepweedsx/test_set_labels.csv')
train_data['Species']=train_data['Species'].astype(str)
train_data


In [ ]:
plt.figure(figsize=(10,5))
species_count=np.array(train_data['Species'])
sns.countplot(species_count)
train_data["Species"].value_counts()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , BatchNormalization , Dropout , GlobalAveragePooling2D
import cv2
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

In [ ]:
training_set,validation_set= train_test_split(train_data, test_size=0.2 , random_state=0)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 zoom_range=0.5,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator= train_datagen.flow_from_dataframe(dataframe=training_set,directory='/kaggle/input/deepweedsx/DeepWeeds_Images_256',x_col="Label",y_col="Species",
                                                   target_size=(224,224),batch_size=16,class_mode="categorical")

validation_generator= validation_datagen.flow_from_dataframe(dataframe=validation_set,directory='/kaggle/input/deepweedsx/DeepWeeds_Images_256',x_col="Label",y_col="Species",
                                                   target_size=(224,224),batch_size=16,class_mode='categorical')



In [ ]:
image_size=160
IMG_SIZE=(224,224,3)

base_model=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
base_model.trainable=False



model=Sequential()
model.add(base_model)

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2),padding='same'))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2),padding='same'))


model.add(Flatten())

model.add(Dense(1024,activation='relu'))

model.add(Dense(512,activation='relu'))

model.add(Dense(256,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(32,activation='relu'))

model.add(Dense(9,activation='softmax'))


opt=keras.optimizers.Adam(learning_rate=0.001,epsilon=1e-07)
model.compile(optimizer="sgd",loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
epochs=50
steps_per_epochs=165
validation_step=165
Callback= tf.keras.callbacks.ModelCheckpoint(filepath='/output/kaggle/working',monitor='val_accuracy', verbose= 1 ,save_best_only=True,)

history=model.fit_generator(train_generator,epochs=epochs,
                            steps_per_epoch=steps_per_epochs,
                            validation_data=validation_generator,
                            validation_steps=validation_step,callbacks=[Callback])

In [ ]:
epochs=[i for i in range(50)]
fig,ax=plt.subplots(1,2)
train_acc=history.history['accuracy']
train_loss=history.history['loss']
val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']

fig.set_size_inches(20,10)
ax[0].plot(epochs,train_acc,'go-',label='Training Accuracy')
ax[0].plot(epochs,val_acc,'ro-',label='validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")


ax[1].plot(epochs,train_loss,'go-',label='Training loss')
ax[1].plot(epochs,val_loss,'ro-',label='validation loss')
ax[1].set_title('Training & Validation loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("loss")
plt.show()